### Inventory of stock items, not managed for a month.

In [1]:
import pandas as pd

In [68]:
# gsshop=pd.read_pickle('data/GSS_SELL.pk')
# oliveYoung=pd.read_pickle('data/OLI_SELL.pk')
# hmall=pd.read_pickle('data/HMA_SELL.pk')

gsshop=pd.read_pickle('data/GSS0713.pk')
# oliveYoung=pd.read_pickle('data/OLI0713.pk')
# hmall=pd.read_pickle('data/HMA0713.pk')

In [75]:
data = gsshop;print(data.shape)
data.head(1)

(2314693, 18)


,SELL_AMOUNT,STOCK_AMOUNT,REVISE_STOCK_AMOUNT,STOCK_ID,ITEM_ID,UPT_DT,COLLECT_DAY,UPT_ID,REG_DT,ITEM_NUM,GOODS_NAME,COLLECT_URL,NORMAL_PRICE_SITE,BRAND_NAME,SITE_NO,CATE_NAMES,BRAND,SELL
0,0,726,726,16437651001,115022,2018-07-13 13:36:16,2018-06-01,FILTER ALGO,2018-06-01,16437651,[바비리스] 1600W 이온 헤어드라이기 접이식/2단/쿨샷 5383K +,http://www.gsshop.com/prd/prd.gs?prdid=16437651,27800,바비리스,1,바디/헤어>헤어케어,init,0


## SELL_AMOUNT == 0 찾기

In [76]:
Q_SALES = data.groupby('ITEM_ID').SELL_AMOUNT.sum().reset_index()
Q_SALES_ZERO = Q_SALES[Q_SALES.SELL_AMOUNT==0]
print(Q_SALES.shape,Q_SALES_ZERO.shape)
Q_SALES_ZERO.head(1)

(49620, 2) (28626, 2)


,ITEM_ID,SELL_AMOUNT
11,8045,0.0


In [77]:
DATA_ZERO = \
data[data.ITEM_ID.isin(Q_SALES_ZERO.ITEM_ID
                      )][['ITEM_ID','STOCK_ID','SELL_AMOUNT','STOCK_AMOUNT','COLLECT_DAY','BRAND','CATE_NAMES']]
DATA_ZERO.shape

(926072, 7)

## STOCK_AMOUNT == 9999 찾기

In [78]:
DATA_ZERO.loc[DATA_ZERO['STOCK_AMOUNT'].isna(),'STOCK_AMOUNT']=0
a = DATA_ZERO.groupby('ITEM_ID').STOCK_AMOUNT.nunique()
DATA_ZERO= DATA_ZERO[DATA_ZERO.ITEM_ID.isin(a[a==1].reset_index().ITEM_ID.tolist())]
DATA_ZERO['STOCK_AMOUNT'] = DATA_ZERO['STOCK_AMOUNT'].astype('int')
DATA_ZERO['STOCK_AMOUNT'] = DATA_ZERO['STOCK_AMOUNT'].astype('str')
DATA_ZERO_VALUE_999 = DATA_ZERO[DATA_ZERO.STOCK_AMOUNT.str.contains('^99+$')]
print(DATA_ZERO.shape,DATA_ZERO_VALUE_999.shape) 
#(212026, 6) (15127, 6)  : ^99+$
# (212026, 6) (20354, 6) : ^99+
DATA_ZERO_VALUE_999.head(10)



(7208, 7) (984, 7)


,ITEM_ID,STOCK_ID,SELL_AMOUNT,STOCK_AMOUNT,COLLECT_DAY,BRAND,CATE_NAMES
5808,12834191,31388300001,0,999,2018-06-30,init,스킨케어>클렌저
11170,12276244,31496636001,0,99,2018-06-13,init,향수>남녀공용향수
11394,530499,23720935001,0,9999,2018-06-23,init,스킨케어>베이직 기초
15935,12625445,31670833001,0,999,2018-06-26,init,스킨케어>베이직 기초
16777,5312607,28128832001,0,9999,2018-06-25,LOREAL,바디/헤어>바디모이스처라이저
18989,97545,26515876001,0,9999,2018-06-29,init,미분류>미분류
21996,12178871,31502439001,0,999,2018-06-10,init,스킨케어>마스크/팩
24892,12625490,31693355001,0,999,2018-06-26,LG,스킨케어>남성케어
26987,12595143,14453982001,0,999,2018-06-24,init,스킨케어>베이직 기초
29027,12046151,31414959001,0,999,2018-06-06,init,스킨케어>클렌저


In [79]:
DATA_ZERO_VALUE_999.ITEM_ID.nunique() # TOTAL AMOUNT 

692

### 브랜드별(AP,LG,LVMH,LOREAL) 999case item_id 5개씩 보자

In [80]:
print(
DATA_ZERO_VALUE_999[DATA_ZERO_VALUE_999.BRAND=='AP'].ITEM_ID.unique()[:10],
DATA_ZERO_VALUE_999[DATA_ZERO_VALUE_999.BRAND=='LG'].ITEM_ID.unique()[:10],
DATA_ZERO_VALUE_999[DATA_ZERO_VALUE_999.BRAND=='LOREAL'].ITEM_ID.unique()[:10],
DATA_ZERO_VALUE_999[DATA_ZERO_VALUE_999.BRAND=='LVMH'].ITEM_ID.unique()[:10]
)

[8750156 12721985 12721996 12794053 12722000 12756680 12597033 12609406
 1445434 12733865] [12625490 12625344 12180291 12625467 12625354 12625461 12392147 12625393
 12625392 12755019] [5312607 5312588 5319995 5312598 5312606 4338237 5319964 4337644 5320003
 1760318] [12722022 12625534 12625535 12834127 12834129 12625253 6672884 12834128
 12834130]


# [ALL] Inventory by Brand & CATE

In [152]:
# IVT_GS = data.groupby(['CATE_NAMES','BRAND']).ITEM_ID.nunique().reset_index().pivot(
#     index='CATE_NAMES',columns='BRAND',values='ITEM_ID')
# IVT_GS['SITE_NAME']='GSSHOP'
print(IVT_GS.shape);IVT_GS.head(1)

(28, 6)


BRAND,AP,LG,LOREAL,LVMH,init,SITE_NAME
CATE_NAMES,,,,,,
기획세트>기획세트,117.0,193.0,45.0,4.0,128.0,GSSHOP


In [151]:
# print('total item id :',data.ITEM_ID.nunique())
# # print('total item id by brand :',data.groupby('BRAND').ITEM_ID.nunique())
# # DATA_ZERO_VALUE_999.groupby('BRAND').ITEM_ID.nunique() # size()
# IVT_HML = data.groupby(['CATE_NAMES','BRAND']).ITEM_ID.nunique().reset_index().pivot(
#     index='CATE_NAMES',columns='BRAND',values='ITEM_ID')
# IVT_HML['SITE_NAME']='HMALL'
print(IVT_HML.shape);IVT_HML.head(1)

(28, 6)


BRAND,AP,LG,LOREAL,LVMH,init,SITE_NAME
CATE_NAMES,,,,,,
기획세트>기획세트,27.0,125.0,10.0,1.0,105.0,HMALL


In [9]:
IVT_OLI = data.groupby(['CATE_NAMES','BRAND']).ITEM_ID.nunique().reset_index().pivot(
    index='CATE_NAMES',columns='BRAND',values='ITEM_ID')
IVT_OLI['SITE_NAME']='OLIVE'
print(IVT_OLI.shape);IVT_OLI.head(1)

(28, 6)


BRAND,AP,LG,LOREAL,LVMH,init,SITE_NAME
CATE_NAMES,,,,,,
기획세트>기획세트,4.0,27.0,14.0,NaN,113.0,OLIVE


In [164]:
TOTAL_IVT = pd.concat([
    IVT_GS.reset_index(),
    IVT_HML.reset_index(),
    IVT_OLI.reset_index()
])[['CATE_NAMES','SITE_NAME','AP','LG','LOREAL','LVMH']]
TOTAL_IVT=TOTAL_IVT.fillna(0)
TOTAL_IVT.sort_values(by=['CATE_NAMES','SITE_NAME']).set_index('CATE_NAMES').to_csv('data/countAll.csv')
TOTAL_IVT.sort_values(by=['CATE_NAMES','SITE_NAME']).set_index('CATE_NAMES')

BRAND,SITE_NAME,AP,LG,LOREAL,LVMH
CATE_NAMES,,,,,
기획세트>기획세트,GSSHOP,117.0,193.0,45.0,4.0
기획세트>기획세트,HMALL,27.0,125.0,10.0,1.0
기획세트>기획세트,OLIVE,4.0,27.0,14.0,0.0
메이크업>기타,GSSHOP,8.0,5.0,2.0,15.0
메이크업>기타,HMALL,5.0,6.0,9.0,9.0
메이크업>기타,OLIVE,0.0,0.0,0.0,1.0
메이크업>네일,GSSHOP,11.0,1.0,9.0,2.0
메이크업>네일,HMALL,23.0,0.0,0.0,3.0
메이크업>네일,OLIVE,0.0,1.0,0.0,0.0


# [999case] Inventory by Brand & CATE

In [11]:
DATA_ZERO_VALUE_999_pivot= \
DATA_ZERO_VALUE_999.groupby(['CATE_NAMES','BRAND']).ITEM_ID.nunique().reset_index().pivot(
    index='CATE_NAMES',columns='BRAND',values='ITEM_ID')
DATA_ZERO_VALUE_999_pivot=DATA_ZERO_VALUE_999_pivot.fillna(0)

In [97]:
# DATA_ZERO_VALUE_999_pivot['SITE_NAME']='GSSHOP'
# DATA_GS = DATA_ZERO_VALUE_999_pivot
print(DATA_GS.shape);DATA_GS.head(2)

(28, 6)


BRAND,AP,LG,LOREAL,LVMH,init,SITE_NAME
CATE_NAMES,,,,,,
기획세트>기획세트,3.0,30.0,0.0,1.0,15.0,GSSHOP
메이크업>기타,0.0,1.0,0.0,1.0,4.0,GSSHOP


In [102]:
# DATA_ZERO_VALUE_999_pivot['SITE_NAME']='HMALL'
# DATA_HML = DATA_ZERO_VALUE_999_pivot
print(DATA_HML.shape);DATA_HML.tail(2)

(28, 6)


BRAND,AP,LG,LOREAL,LVMH,init,SITE_NAME
CATE_NAMES,,,,,,
향수>여성향수,0.0,0.0,0.0,2.0,14.0,HMALL
향수>홈프래그런스,0.0,0.0,0.0,0.0,200.0,HMALL


In [12]:
DATA_ZERO_VALUE_999_pivot['SITE_NAME']='OLIVE'
DATA_OLI = DATA_ZERO_VALUE_999_pivot
print(DATA_OLI.shape);DATA_OLI.head(8)

(8, 3)


BRAND,LG,init,SITE_NAME
CATE_NAMES,,,
메이크업>네일,0.0,1.0,OLIVE
메이크업>툴/소품,0.0,1.0,OLIVE
바디/헤어>덴탈케어,0.0,4.0,OLIVE
스킨케어>남성케어,0.0,1.0,OLIVE
스킨케어>마스크/팩,0.0,2.0,OLIVE
스킨케어>베이직 기초,1.0,0.0,OLIVE
스킨케어>선케어,0.0,1.0,OLIVE
스킨케어>클렌저,0.0,1.0,OLIVE


In [113]:
DATA_OLI= DATA_HML[['AP','LG','LOREAL','LVMH','init']]
DATA_OLI[['AP','LG','LOREAL','LVMH','init']]=0
DATA_OLI['SITE_NAME']='OLIVE'
print(DATA_OLI.shape);DATA_OLI.head(2)

(28, 6)


c:\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\python36\lib\site-packages\pandas\core\frame.py:2544: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
c:\python36\lib\site-packages\pandas\core\frame.py:2514: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_array(key, value)


BRAND,AP,LG,LOREAL,LVMH,init,SITE_NAME
CATE_NAMES,,,,,,
기획세트>기획세트,0.0,0.0,0.0,0.0,0.0,OLIVE
메이크업>기타,0.0,0.0,0.0,0.0,0.0,OLIVE


In [163]:
TOTAL = pd.concat([
    DATA_GS.reset_index(),
    DATA_HML.reset_index(),
    DATA_OLI.reset_index()
])[['CATE_NAMES','SITE_NAME','AP','LG','LOREAL','LVMH']]
TOTAL=TOTAL.fillna(0)
TOTAL.sort_values(by=['CATE_NAMES','SITE_NAME']).set_index('CATE_NAMES').to_csv('data/count999.csv')
TOTAL.sort_values(by=['CATE_NAMES','SITE_NAME']).set_index('CATE_NAMES')

BRAND,SITE_NAME,AP,LG,LOREAL,LVMH
CATE_NAMES,,,,,
기획세트>기획세트,GSSHOP,3.0,30.0,0.0,1.0
기획세트>기획세트,HMALL,1.0,42.0,0.0,0.0
기획세트>기획세트,OLIVE,0.0,0.0,0.0,0.0
메이크업>기타,GSSHOP,0.0,1.0,0.0,1.0
메이크업>기타,HMALL,0.0,0.0,0.0,0.0
메이크업>기타,OLIVE,0.0,0.0,0.0,0.0
메이크업>네일,GSSHOP,0.0,0.0,0.0,0.0
메이크업>네일,HMALL,9.0,0.0,0.0,1.0
메이크업>네일,OLIVE,0.0,0.0,0.0,0.0


# Total Revenue by Brand

In [48]:
# REV_BY_ITEM_BRAND = data.groupby(['ITEM_ID','BRAND']).SELL.sum().reset_index()
# REV_BY_BRAND = REV_BY_ITEM_BRAND.groupby('BRAND').SELL.sum().reset_index()

,BRAND,SELL
0,AP,3.099245e+08
1,LG,2.724368e+08
2,LOREAL,2.896145e+08
3,LVMH,6.284245e+07
4,init,1.467808e+09


In [51]:
REV_BY_BRAND= data.groupby('BRAND').SELL.sum().reset_index()
REV_BY_BRAND.SELL = REV_BY_BRAND.SELL.astype('str')
REV_BY_BRAND.T

,0,1,2,3,4
BRAND,AP,LG,LOREAL,LVMH,init
SELL,309924540.0,272436850.0,289614530.0,62842450.0,1467807920.0


# Total Revenue by Brand & CATE

In [74]:
REV_BY_BRAND= data.groupby(['CATE_NAMES','BRAND']).SELL.sum().reset_index().pivot(
index='CATE_NAMES',columns='BRAND',values='SELL'
)
REV_BY_BRAND = REV_BY_BRAND.fillna(0)
# REV_BY_BRAND.SELL = REV_BY_BRAND.SELL.astype('str')
REV_BY_BRAND.head()

BRAND,AP,LG,LOREAL,LVMH,init
CATE_NAMES,,,,,
기획세트>기획세트,77841100.0,73586800.0,43516600.0,45900.0,131044560.0
메이크업>기타,243000.0,191800.0,675100.0,2994500.0,8328950.0
메이크업>네일,181700.0,0.0,454400.0,0.0,22986680.0
메이크업>립,11293950.0,611400.0,11612500.0,14003850.0,54669220.0
메이크업>베이스,68089240.0,14904450.0,29151050.0,9878400.0,169402680.0


In [166]:
#Series 의 컬럼이름말고 값만 가져오고 싶을 때 : 데이터 형식은 array!
REV_BY_BRAND['init'].values

array([  1.31044560e+08,   8.32895000e+06,   2.29866800e+07,
         5.46692200e+07,   1.69402680e+08,   2.91799000e+07,
         1.52761300e+07,   8.85215000e+06,   1.78964300e+07,
         1.38560000e+06,   5.79200000e+05,   3.46223600e+07,
         3.36191100e+07,   3.90211500e+07,   7.72975000e+06,
         1.85094920e+08,   6.30205000e+06,   2.22687300e+07,
         7.14004500e+07,   4.80470000e+06,   3.85258740e+08,
         8.72778900e+07,   7.12279700e+07,   2.48460000e+06,
         3.94879000e+07,   1.25120000e+06,   1.52700000e+07,
         1.08490000e+06])